In [1]:
import torch
from torch.optim import SGD
from data import NYTimes, Hospital_US
import pickle
import threading
import numpy
data1 = NYTimes(level='states')
N = 4e7
us_state = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
states=us_state.values()

In [8]:
beta = torch.tensor(1e-1, dtype=torch.float, requires_grad=True)
gamma = torch.tensor(1e-1, dtype=torch.float, requires_grad=True)
sigma = torch.tensor(1e-1, dtype=torch.float, requires_grad=True)
mu = torch.tensor(1e-3, dtype=torch.float, requires_grad=True)
alpha = torch.tensor(2e-1, dtype=torch.float, requires_grad=True)
rho = torch.tensor(1e-1, dtype=torch.float, requires_grad=True)
theta = torch.tensor(1e-2, dtype=torch.float, requires_grad=True)
a=torch.zeros(50,3,1,dtype=torch.float,requires_grad=True)
s0 = torch.tensor(1, dtype=torch.float, requires_grad=True)
e0 = torch.tensor(1e1, dtype=torch.float, requires_grad=True)
i0 = torch.tensor(1, dtype=torch.float, requires_grad=True)

optimizer = SGD([
    {'params': beta, 'lr': 1e-5},
    {'params': gamma, 'lr': 1e-5},
    {'params': sigma, 'lr': 1e-5},
    {'params': mu, 'lr': 1e-8},
    {'params': alpha, 'lr': 1e-5},
    {'params': s0, 'lr': 1e-4},
    {'params': e0, 'lr': 1e-4},
    {'params': i0, 'lr': 1e-4},
    {'params':a,'lr':1e-4}
])
All_E=[None]*50

def train_state(state_num,features):
    features=torch.from_numpy(features)
    confirm, death = data1.get('2020-03-31', '2020-06-01', states[state_num].lower()) 
   
    size = len(confirm)
    S = [torch.tensor(0) for _ in range(size)]
    E = [torch.tensor(0) for _ in range(size)]
    I = [torch.tensor(0) for _ in range(size)]
    R = [torch.tensor(0) for _ in range(size)]
    deltas=[torch.tensor(0) for _ in range(2500)]
    deltas=deltas.view(50,50)
    for j in range(1000):
        deltas=torch.matmul(a[state_num], features)
        print(deltas)
        optimizer.zero_grad()
        S[0], E[0], I[0], R[0] = s0 * N, e0 * confirm[0], i0 * confirm[0], (1 - i0) * confirm[0]
        All_E[state_num][0]=E[0]
        

        loss = 0
        smooth = 0
        for i in range(size - 1):
            # go for next
            S[i+1] = S[i] - beta * S[i] * (E[i] + I[i]) / N
            connection_sum=0
            for j in range(0,50):
                connection_sum+=deltas[state_num][j]*All_E[j][i]
            E[i+1] = connection_sum+E[i] + beta * S[i] * (E[i] + I[i]) / N - sigma * E[i]
            All_E[state_num][i+1]=E[i+1]
            I[i+1] = I[i] + mu * sigma * E[i] - gamma * I[i]
            R[i+1] = R[i] + gamma * I[i]
            
            loss += torch.square(1 - (I[i+1] + R[i+1]) / (confirm[i + 1]))
        loss /= size
        loss.backward()
        optimizer.step()
        file=open(str(states[state_num])+".txt","a")
        if j % 100 == 99:
            #print('=' * 10, 'Round {} '.format(j), '=' * 10)
            file.write("Epoch"+str(j)+"for state"+states[state_num]+"\n")
            file.write(str(loss.item())+"\n")
            """
            print('beta: ', beta.item(), beta.grad.item())
            print('gamma: ', gamma.item(), gamma.grad.item())
            print('sigma: ', sigma.item(), sigma.grad.item())
            print('mu: ', mu.item(), mu.grad.item())
            print('alpha: ', alpha.item(), alpha.grad.item())
            print('rho: ', rho.item(), rho.grad.item())
            print('theta: ', theta.item(), theta.grad.item())
            print('s0: ', s0.item(), s0.grad.item())
            print('e0: ', e0.item(), e0.grad.item())
            print('i0: ', i0.item(), i0.grad.item())
            """
threads=[None]*50
arr_pickle = open ("ati_dict.pkl", "rb")
features = pickle.load(arr_pickle)
for i in range(0,50):
    
    threads[i] = threading.Thread(target=train_state, args=(i,features[i]))

========== Round 99  ==========
1.7476648092269897
beta:  0.09943939745426178 -0.24212078750133514
gamma:  0.16106724739074707 -12.402329444885254
sigma:  0.09947877377271652 0.26176854968070984
mu:  0.000888978538569063 1.8765711784362793
alpha:  0.12382887303829193 30.725955963134766
rho:  0.1680762767791748 -15.663971900939941
theta:  0.014358984306454659 -3.5835812091827393
s0:  0.9994378089904785 -0.024457836523652077
e0:  9.999905586242676 0.00024862028658390045
i0:  0.8688209652900696 4.378199577331543
========== Round 199  ==========
0.9822046160697937
beta:  0.09973875433206558 -0.327766090631485
gamma:  0.16798990964889526 -3.6379382610321045
sigma:  0.09927191585302353 0.17002896964550018
mu:  0.000899355043657124 -17.395658493041992
alpha:  0.10172269493341446 16.338611602783203
rho:  0.1775457262992859 -5.700092792510986
theta:  0.017560424283146858 -2.905764102935791
s0:  0.999739944934845 -0.033107202500104904
e0:  9.999905586242676 -0.001440686290152371
i0:  0.839584410